In [1]:
import requests

def callgpt(prompt, query):
    url = 'https://gcrendpoint.azurewebsites.net/openai/deployments/gpt-4-32k/chat/completions?api-version=2023-03-15-preview'
    api_key = 'oaip_qKIbzuMTnmjWfbUgSwXVFOvgZgTWmaGp'  
    headers = {'Content-Type': 'application/json', 'api-key': api_key}  
    data = {
        "messages": [  
            {"role": "system", "content": prompt},  
            {"role": "user", "content": query},
        ],
        "max_tokens": 4000,
        "temperature": 0.7,
        "n":1
    }

    resp = requests.post(url, json=data, headers=headers)
    return resp.json()

In [4]:
import json  
  
# Path to your .jsonl file  
jsonl_file_path = 'multiagent-pytorch.json'
  
# Initialize an empty list to store the JSON objects  
data = []  
  
# Open the file and read line by line  
with open(jsonl_file_path, 'r', encoding='utf-8') as file:  
    lines = file.readlines()
    # Parse each line as a JSON object and append to the list  
    data = json.loads('\n'.join(lines))


In [5]:
data[2]['content']

'Creating a deep learning framework like PyTorch is a massive undertaking that involves many components such as tensor operations, automatic differentiation, neural network layers, optimizers, and utilities for data loading and model saving. Below is a simplified, high-level architecture for such a framework.\n\nFiles:\n- tensor.py\n- autograd.py\n- module.py\n- layers.py\n- optim.py\n- utils.py\n- dataloader.py\n- main.py\n\nAPIs:\ntensor.py:\n```python\nclass Tensor:\n    def __init__(self, data, requires_grad=False):\n        # Initializes the tensor with data and a boolean indicating if it requires gradient computation\n\n    def __add__(self, other):\n        # Overloads the addition operator\n\n    def __mul__(self, other):\n        # Overloads the multiplication operator\n\n    def backward(self):\n        # Performs backpropagation starting from this tensor\n\n    def zero_grad(self):\n        # Zeros the gradient of the tensor\n```\n\nautograd.py:\n```python\nclass Function:\n

In [6]:
file_apis = {}
parsing_code = False
code = ""
for line in data[2]['content'].split('\n'):
    if '.py' in line:
        if '-' in line:
            continue
        filename = line.split(":")[0]
        parsing_code = True
        continue
    if parsing_code:
        code = code + '\n' + line
        if line == '```':
            file_apis[filename] = code
            parsing_code = False
print(file_apis['tensor.py'])
        


```python
class Tensor:
    def __init__(self, data, requires_grad=False):
        # Initializes the tensor with data and a boolean indicating if it requires gradient computation

    def __add__(self, other):
        # Overloads the addition operator

    def __mul__(self, other):
        # Overloads the multiplication operator

    def backward(self):
        # Performs backpropagation starting from this tensor

    def zero_grad(self):
        # Zeros the gradient of the tensor
```


In [7]:
from jinja2 import Template
def round1(file_apis):
    system = """
NOTICE
Role: You are a professional engineer; the main goal is to write PEP8 compliant, elegant, modular, easy to read and maintain Python 3.9 code (but you can also use other programming language)
ATTENTION: Use '##' to SPLIT SECTIONS, not '#'. Output format carefully referenced "Format example".

## Code: Write code with triple quoto, based on the following list and context.
1. Do your best to implement THIS ONLY ONE FILE. ONLY USE EXISTING API. IF NO API, IMPLEMENT IT.
2. Requirement: Based on the context, implement one following code file, note to return only in code form, your code will be part of the entire project, so please implement complete, reliable, reusable code snippets
3. Attention1: If there is any setting, ALWAYS SET A DEFAULT VALUE, ALWAYS USE STRONG TYPE AND EXPLICIT VARIABLE.
4. Attention2: YOU MUST FOLLOW "Data structures and interface definitions". DONT CHANGE ANY DESIGN.
5. Think before writing: What should be implemented and provided in this document?
6. CAREFULLY CHECK THAT YOU DONT MISS ANY NECESSARY CLASS/FUNCTION IN THIS FILE.
7. Do not use public member functions that do not exist in your design.
"""
    user_template = Template("""
    Files:
    {{filenames}}

    APIs:
    {{apis}}

    You only need to implement {{implement}}. Implement all functions and additional functions you need. DO NOT LET ME TO IMPLEMENT ANYTHING!!!!
    Make sure your response code is runnable.
    Do not response any content in {{otherfiles}}. Strictly follow the response format. Do not answer any other content or suggestions.
    """)
    filenames = '\n'.join(file_apis.keys())
    apis = ''
    for file in file_apis:
        apis = apis + file + '\n' + file_apis[file]
    queries = {}
    responses = {}
    for file in file_apis:
        otherfiles = [_ for _ in file_apis.keys() if _ != file]
        user_msg = user_template.render({"filenames":filenames, "apis":apis, 'implement': file, 'otherfiles':otherfiles})
        queries[file] = {'system': system, 'user_msg': user_msg}
        responses[file] = callgpt(system, user_msg)
        print(file, responses[file])
    return queries, responses


In [8]:
r1_queries, r1_responses = round1(file_apis)


tensor.py {'id': 'chatcmpl-8SlBo4UQG6qxHEpqWzpQ9kHKYpS60', 'object': 'chat.completion', 'created': 1701865272, 'model': 'gpt-4-32k', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```python\nimport numpy as np\n\nclass Tensor:\n\n    def __init__(self, data: np.array, requires_grad: bool = False):\n        self.data = np.array(data, dtype=np.float64)\n        self.requires_grad = requires_grad\n        self._grad = None\n        self._grad_fn = None\n\n    @property\n    def shape(self):\n        return self.data.shape\n\n    @property\n    def grad(self):\n        return self._grad\n\n    @property\n    def grad_fn(self):\n        if not self.requires_grad:\n            raise Exception("This tensor is not backpropagation compatible")\n        return self._grad_fn\n\n    def backward(self, grad=None):\n        if not self.grad_fn:\n            return False\n\n        if grad is None and self._grad is None:\n            # in case if this is last loss tensor\n     

In [9]:
print(r1_responses['tensor.py']['choices'][0]['message']['content'])

```python
import numpy as np

class Tensor:

    def __init__(self, data: np.array, requires_grad: bool = False):
        self.data = np.array(data, dtype=np.float64)
        self.requires_grad = requires_grad
        self._grad = None
        self._grad_fn = None

    @property
    def shape(self):
        return self.data.shape

    @property
    def grad(self):
        return self._grad

    @property
    def grad_fn(self):
        if not self.requires_grad:
            raise Exception("This tensor is not backpropagation compatible")
        return self._grad_fn

    def backward(self, grad=None):
        if not self.grad_fn:
            return False

        if grad is None and self._grad is None:
            # in case if this is last loss tensor
            grad = self.__class__(1., requires_grad=False)

        elif self.grad is not None:
            grad = self.grad

        if not self.requires_grad:
            raise Exception("This tensor is not backpropagation compatible")



In [10]:
def round2(file_codes):
    system = """
You are a professional software engineer, and your main task is to review the code. You need to ensure that the code conforms to the PEP8 standards, is elegantly designed and modularized, easy to read and maintain, and is written in Python 3.9 (or in another programming language).
ATTENTION: Use '##' to SPLIT SECTIONS, not '#'. Output format carefully referenced "Format example".

## Code Review: Based on the following context and code, and following the check list, Provide key, clear, concise, and specific code modification suggestions, up to 5.
```
1. Check 0: Is the code implemented as per the requirements?
2. Check 1: Are there any issues with the code logic?
3. Check 2: Does the existing code follow the "Data structures and interface definitions"?
4. Check 3: Is there a function in the code that is omitted or not fully implemented that needs to be implemented?
5. Check 4: Does the code have unnecessary or lack dependencies?

## Format example

## Code Review
1. The code ...
2. ...
3. ...
4. ...
5. ...
"""
    user_template = Template("""
    Task: Write a deep learning framework like PyTorch 

    Codebase:
    {{codebase}}

    Based on the codebase, you only need to give advice on {{file}}.
    Do not give advice any content in {{otherfiles}}. Strictly follow the response format.
    Do not answer any other content or suggestions.
    """)
    codebase = ''
    queries = {}
    responses = {}
    
    for file in file_codes:
        codebase = codebase +'\n' + file + '\n' + file_codes[file]
    for file in file_apis:
        otherfiles = [_ for _ in file_apis.keys() if _ != file]
        user_msg = user_template.render({"codebase": codebase, 'file': file, 'otherfiles':otherfiles})
        queries[file] = {'system': system, 'user_msg': user_msg}        
        responses[file] = callgpt(system, user_msg)
        print(file, responses[file])
    return queries, responses

In [11]:
file_codes_r1 = {}
for file in file_apis:
    file_codes_r1[file] = r1_responses[file]['choices'][0]['message']['content']
r2_queries, r2_responses = round2(file_codes_r1)

tensor.py {'id': 'chatcmpl-8SlEG4a0xk6RimGI4yRtOMpSgTilq', 'object': 'chat.completion', 'created': 1701865424, 'model': 'gpt-4-32k', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '## Code Review\n1. The code in tensor.py does not seem to be implemented as per the requirements. The code does not follow the object-oriented programming standards for encapsulation. For instance, the data attribute of the Tensor class is directly accessed and modified in the __add__ and __mul__ methods, which might lead to unexpected behavior and bugs.\n2. There are some issues with the code logic. The __add__ and __mul__ methods are modifying the data attribute of the Tensor object in-place, which could lead to unexpected results. In addition, the _make_grad_fn method returns a function for the gradient computation, but it does not assign this function to the _grad_fn attribute, so the backward method will not work as expected.\n3. The existing code does not fully follow the "Data st

In [12]:
def round3(file_codes, file_reviews):
    system = """
NOTICE
Role: You are a professional engineer; the main goal is to write PEP8 compliant, elegant, modular, easy to read and maintain Python 3.9 code (but you can also use other programming language)
ATTENTION: Use '##' to SPLIT SECTIONS, not '#'. Output format carefully referenced "Format example".

## Code: Write code with triple quoto, based on the following list and context.
1. Do your best to implement THIS ONLY ONE FILE. ONLY USE EXISTING API. IF NO API, IMPLEMENT IT.
2. Requirement: Based on the context, implement one following code file, note to return only in code form, your code will be part of the entire project, so please implement complete, reliable, reusable code snippets
3. Attention1: If there is any setting, ALWAYS SET A DEFAULT VALUE, ALWAYS USE STRONG TYPE AND EXPLICIT VARIABLE.
4. Attention2: YOU MUST FOLLOW "Data structures and interface definitions". DONT CHANGE ANY DESIGN.
5. Think before writing: What should be implemented and provided in this document?
6. CAREFULLY CHECK THAT YOU DONT MISS ANY NECESSARY CLASS/FUNCTION IN THIS FILE.
7. Do not use public member functions that do not exist in your design.
"""
    user_template = Template("""
    Task: Write a deep learning framework like PyTorch 

    Codebase:
    {{codebase}}
    
    Reviews:
    {{review}}

    Based on the codebase, and review comments, revise your past implementation of {{file}}. 
    You only need to implement {{file}}. Implement all functions and additional functions you need. DO NOT LET ME TO IMPLEMENT ANYTHING!!!!
    Make sure your response code is runnable.
    Do not response any content in {{otherfiles}}. Strictly follow the response format. Do not answer any other content or suggestions.
    """)
    codebase = ''
    reviews = ''
    queries = {}
    responses = {}
    
    for file in file_codes:
        codebase = codebase +'\n' + file + '\n' + file_codes[file]
        # reviews = reviews + '\n' + file + '\n' + file_reviews[file]
    reviews = file_reviews[file]
        
    for file in file_apis:
        otherfiles = [_ for _ in file_apis.keys() if _ != file]
        user_msg = user_template.render({"codebase": codebase, 'review': reviews, 'file': file, 'otherfiles':otherfiles})
        queries[file] = {'system': system, 'user_msg': user_msg}        
        responses[file] = callgpt(system, user_msg)
        print(file, responses[file])
    return queries, responses

In [13]:
reviews_r2 = {}
for file in file_apis:
    reviews_r2[file] = r2_responses[file]['choices'][0]['message']['content']
r3_queries, r3_responses = round3(file_codes_r1, reviews_r2)

tensor.py {'id': 'chatcmpl-8SlGP87wbQabHafKcNQsoVb5Kf3LF', 'object': 'chat.completion', 'created': 1701865557, 'model': 'gpt-4-32k', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```python\n## Revised tensor.py\n\nimport numpy as np\n\nclass Tensor:\n\n    def __init__(self, data: np.array, requires_grad: bool = False):\n        self.data = np.array(data, dtype=np.float64)\n        self.requires_grad = requires_grad\n        self._grad = None\n        self._grad_fn = None\n\n    @property\n    def shape(self):\n        return self.data.shape\n\n    @property\n    def grad(self):\n        return self._grad\n\n    @property\n    def grad_fn(self):\n        if not self.requires_grad:\n            raise Exception("This tensor is not backpropagation compatible")\n        return self._grad_fn\n\n    def backward(self, grad=None):\n        if not self.grad_fn:\n            return False\n\n        if grad is None and self._grad is None:\n            # in case if this is

In [14]:
log = [{'r1_queries': r1_queries, 'r1_responses': r1_responses}, 
       {'r2_queries': r2_queries, 'r2_responses': r2_responses}, 
       {'r3_queries': r3_queries, 'r3_responses': r3_responses}]

# Open the file in write mode  
with open('log_3_round.jsonl', 'w') as file:  
    # Iterate over the list of dictionaries  
    for d in log:  
        # Convert the dictionary to a JSON string  
        json_str = json.dumps(d)  
        # Write the JSON string to the file with a newline  
        file.write(json_str + '\n')